# How Hofestede's dimensions correlate to the economy

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mpld3
import plotly.express as px
import chart_studio.plotly as py
import chart_studio

In [7]:
econdata=pd.read_csv("/Users/charan/Documents/website/hofestedes_dimensions_v_economy/gdp_oecd.csv")
hofdata=pd.read_csv("/Users/charan/Documents/website/hofestedes_dimensions_v_economy/6-dimensions-for-website-2015-08-16 (1).csv",sep=";")
codetoname=pd.read_csv("/Users/charan/Documents/website/hofestedes_dimensions_v_economy/countrycode_to_name_key - Sheet1.csv")


In [8]:
econdata.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,GDP,TOT,MLN_USD,A,1960,25073.260,NaN
1,AUS,GDP,TOT,MLN_USD,A,1961,25364.881,NaN
2,AUS,GDP,TOT,MLN_USD,A,1962,27955.357,NaN
3,AUS,GDP,TOT,MLN_USD,A,1963,30432.979,NaN
4,AUS,GDP,TOT,MLN_USD,A,1964,32745.289,NaN


In [9]:
hofdata.head()

,ctr,country,pdi,idv,mas,uai,ltowvs,ivr
0,AFE,Africa East,64,27,41,52,32,40
1,AFW,Africa West,77,20,46,54,9,78
2,ALB,Albania,#NULL!,#NULL!,#NULL!,#NULL!,61,15
3,ALG,Algeria,#NULL!,#NULL!,#NULL!,#NULL!,26,32
4,AND,Andorra,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,65


In [10]:
hofdata.shape

(111, 8)

In [11]:
codetoname.head()

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN


In [12]:
codetoname.shape

(249, 11)

In [13]:
count=0
for i in range(hofdata.shape[0]):
    a = hofdata["country"].loc[i]
    for j in range(codetoname.shape[0]):
        b = codetoname["name"].loc[j]
        if(a==b):
            hofdata["ctr"].loc[i]=codetoname["alpha-3"].loc[j]

In [14]:
uecondata=econdata["LOCATION"].unique()
target1=[]
for i in range(hofdata.shape[0]):
    a = hofdata["ctr"].loc[i]
    #print(a)
    found=False;
    for j in range(uecondata.shape[0]):
        b=uecondata[j]
        #print(b)
        if(a==b):
            found=True;
            break;
    if(not found):
        target1.append(i)
hofdata.drop(labels=target1,inplace=True)

In [28]:
username = 'charan_s' # your username
api_key = 'dS1pbnnqdE3t3yF2DsUu' # your api key - go to profile > settings > regenerate key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

In [33]:
def graphdimen(dim):
    #fig=plt.figure(figsize=(15,6))
    #plt.style.use("fivethirtyeight")
    #plt.figure(figsize=(15,6));
    lochofdata=hofdata.copy()
    lochofdata.reset_index(inplace=True)
    targets = []
    for i in range(lochofdata.shape[0]):
        if(lochofdata.iloc[i].loc[dim]=="#NULL!"):
            targets.append(i)
    lochofdata.drop(labels=targets,inplace=True)
    #print(lochofdata)
    #lochofdata.reset_index(inplace=True)
    gdpval=[]
    where=[]
    for i in range(lochofdata.shape[0]):
        #print(lochofdata["ctr"].iloc[i])
        curedat = econdata[econdata["LOCATION"].str.contains(lochofdata["ctr"].iloc[i])]
        curedat = curedat[curedat["MEASURE"].str.contains("USD_CAP")]
        curedat = curedat[curedat["TIME"]==2013]
        whereval=curedat["LOCATION"].values[0]
        curedat=curedat["Value"]
        where.append(whereval)
        #print(curedat.values)
        gdpval.append(int(curedat))
    ans=pd.DataFrame()
    #gdpval=gdpval[0].values
    lochofdata[dim].astype(int)
    x=lochofdata[dim].astype(int)
    y=gdpval
    print("Correlation: " + str(np.corrcoef(x,y)[0,1]))
    kind=""
    if(dim=="idv"):
        kind="Individuality Index"
    elif(dim=="pdi"):
        kind="Power Distance Index"
    elif(dim=="uai"):
        kind="Uncertainty Avoidance Index"
    elif(dim=="mas"):
        kind="Masculinity Index"
    elif(dim=="ltowvs"):
        kind="Long Term Orientation Index"
    elif(dim=="ivr"):
        kind="Indulgence versus Restraint"
    ans[kind]=x
    ans["GDP per Capita in USD"]=y
    ans["country"]=where
    fig=px.scatter(ans,x=kind,y="GDP per Capita in USD",hover_data=["country"], trendline="ols",title=kind+" graph")
    fig.write_html(dim+".html")
    py.plot(fig, filename = 'testingthingwith'+dim, auto_open=True)
    fig.show()

In [34]:
graphdimen("mas")

Correlation: -0.0013124655696661763


In [17]:
 #plt.title(kind+" Versus Per Capita GDP")
    #plt.ylabel("Per Capita GDP in USD")
    #plt.xlabel(kind+" score")
    #plt.scatter(x,y)
    #coef = np.polyfit(x,y,1)
    #poly1d_fn = np.poly1d(coef) 
    #plt.plot(x,poly1d_fn(x),'--k')
    #html_str = mpld3.fig_to_html(fig)
    #Html_file= open(dim+".html","w")
    #Html_file.write(html_str)
    #Html_file.close()
    #plt.tight_layout()
    #print(wheref)
    #plt.show()
    #fig=px.scatter(x=x,y=y,trendline="ols")
    #plt.plot(x,poly1d_fn(x),'--k')
    #fig.show()